**This notebook provides quality assurance, much of which cannot be covered by cellxgene validate, of AnnData objects towards CELLxGENE curation**

In [ ]:
import anndata as ad
import json
import pandas as pd
import requests
import scanpy as sc
from IPython.display import display
from scipy import sparse
from urllib.parse import quote
from cellxgene_mods import *
from hca import validate_hca

# Loading the AnnData object
**Update the path of the file**<br>
*The sample `valid.h5ad` that is in this repo is subsampled from https://cellxgene.cziscience.com/e/f15e263b-6544-46cb-a46e-e33ab7ce8347.cxg/ with some metadata alterations for the purpose of this notebook*

In [ ]:
file = 'valid.h5ad'

**Load the AnnData object**

In [ ]:
adata = sc.read_h5ad(file)
cxg_labels = True if 'schema_reference' in adata.uns else False
adata

# data

**Check if any matrix should be stored as sparse format but isn't**

In [ ]:
evaluate_sparsity(adata)

**Look for duplicated layers, raw integer counts of reasonable scale (typically ~10<sup>3</sup> or more), etc.**

In [ ]:
evaluate_data(adata)

**Check if any observations have exactly the same raw count vector to identify possible duplication**

In [ ]:
evaluate_dup_counts(adata)

# obsm
**Confirm at least one set of embeddings is present**

In [ ]:
adata.obsm

**View embeddings to identify which matches paper figures**

In [ ]:
cellpop_field = 'cell_type' if cxg_labels else 'cell_type_ontology_term_id'

if f'{cellpop_field}_colors' in adata.uns:
    remove_colors = False
else:
    remove_colors = True

sc.set_figure_params(dpi=100)
for e in adata.obsm:
    sc.pl.embedding(adata, basis=e, color=cellpop_field, legend_loc='on data')
if remove_colors:
    del adata.uns[f'{cellpop_field}_colors']

**Check that the default_embedding value, if defined, is in obsm**

In [ ]:
if 'default_embedding' in adata.uns:
    de = adata.uns['default_embedding']
    if de not in adata.obsm_keys():
        report(f'{de} not in [{",".join(adata.obsm_keys())}]', 'ERROR')
    else:
        report(f'{de} is in [{",".join(adata.obsm_keys())}]', 'GOOD')

# uns
**Check for required uns fields**

In [ ]:
for f in curator_uns_fields:
    if f not in adata.uns:
        report(f'{f} is required', 'ERROR')

**Confirm portal fields are not in uns**

In [ ]:
if not cxg_labels:
    for f in portal_uns_fields:
        if f in adata.uns:
            report(f'{f} should not be present in uns', 'ERROR')

**Check any _colors fields defined in uns**

In [ ]:
evaluate_uns_colors(adata)

**Browse all of uns**

In [ ]:
adata.uns

# obs

In [ ]:
adata.obs.info()

In [ ]:
adata.obs

**Ensure obs schema fields are present & precise, and no conflict with portal fields**

In [ ]:
evaluate_obs_schema(adata.obs, labels=cxg_labels)

# 10x barcode checker
**Checks the barcodes in obs index against 10x barcode lists**<br>
*5' v1 and v2 kits use the same barcode list as 3' v2*<br>

In [ ]:
assay_field = 'assay' if cxg_labels else 'assay_ontology_term_id'

evaluate_10x_barcodes(assay_field, adata.obs)

**Look for general obs field issues, redundant information,** \
**fields that aren't appropriate as gradient (e.g. cluster number),** \
**any categorical fields with more than 200 categories as they may not be useful in the visualization**

In [ ]:
evaluate_obs(adata.obs, full_obs_standards)

**Investigate any fields that may be redundant**

In [ ]:
#adata.obs[['development_stage_ontology_term_id','author_age']].value_counts(dropna=False)

**See if any donors have variable donor-level metadata** \
*disease may need to be removed from donor_fields if a donor contributed healthy and disease tissue within in study* \
*development_stage may need to be removed from donor_fields for longitudinal studies*

In [ ]:
donor_fields = ['donor_id',
                'sex_ontology_term_id',
                'development_stage_ontology_term_id',
                'self_reported_ethnicity_ontology_term_id',
                'disease_ontology_term_id']
if cxg_labels:
    donor_fields = [f.replace('_ontology_term_id','') for f in donor_fields]

donor_df = pd.DataFrame(adata.obs[donor_fields].value_counts()).reset_index()
inconsistencies = donor_df[donor_df.duplicated(subset='donor_id', keep=False) == True].sort_values('donor_id')
if not inconsistencies.empty:
    report('donor metadata inconsistencies', 'ERROR')
    display(inconsistencies)

**Browse the per donor metadata**

In [ ]:
donor_df

**Flag any donors that are curated to a development_stage term that is deprecated**<br>
These need to also be annotated to the latest ontology version and that mapping provided separately

In [ ]:
deprecated_dev = [
    'HsapDv:0000004','HsapDv:0000082','HsapDv:0000235','HsapDv:0000080','HsapDv:0000174','HsapDv:0000256','HsapDv:0000083',
    'HsapDv:0000084','HsapDv:0000081','HsapDv:0000085','HsapDv:0000236','HsapDv:0000086','HsapDv:0000204','HsapDv:0000089',
    'HsapDv:0000088','HsapDv:0000087','HsapDv:0000090','HsapDv:0000092','HsapDv:0000091','HsapDv:0000094','HsapDv:0000093',
    'MmusDv:0000016','MmusDv:0000030','MmusDv:0000037','MmusDv:0000038','MmusDv:0000039','MmusDv:0000040','MmusDv:0000041',
    'MmusDv:0000044','MmusDv:0000045','MmusDv:0000046','MmusDv:0000047','MmusDv:0000048','MmusDv:0000049','MmusDv:0000050',
    'MmusDv:0000051','MmusDv:0000052','MmusDv:0000053','MmusDv:0000054','MmusDv:0000055','MmusDv:0000056','MmusDv:0000057',
    'MmusDv:0000058','MmusDv:0000059','MmusDv:0000061','MmusDv:0000065','MmusDv:0000066','MmusDv:0000067','MmusDv:0000068',
    'MmusDv:0000070','MmusDv:0000071','MmusDv:0000072','MmusDv:0000073','MmusDv:0000074','MmusDv:0000075','MmusDv:0000076',
    'MmusDv:0000096','MmusDv:0000097','MmusDv:0000098','MmusDv:0000099','MmusDv:0000100','MmusDv:0000101','MmusDv:0000102'
]

auto_migrate = {
	'HsapDv:0000004': 'HsapDv:0000005','HsapDv:0000235': 'HsapDv:0000264','HsapDv:0000080': 'HsapDv:0000264',
    'HsapDv:0000174': 'HsapDv:0000273','HsapDv:0000256': 'HsapDv:0000261','HsapDv:0000081': 'HsapDv:0000264',
    'HsapDv:0000085': 'HsapDv:0000271','HsapDv:0000087': 'HsapDv:0000258','HsapDv:0000094': 'HsapDv:0000272',
    'HsapDv:0000093': 'HsapDv:0000227','MmusDv:0000041': 'unknown','MmusDv:0000044': 'MmusDv:0000138',
    'MmusDv:0000045': 'MmusDv:0000140','MmusDv:0000046': 'MmusDv:0000141','MmusDv:0000049': 'MmusDv:0000150',
    'MmusDv:0000050': 'MmusDv:0000151','MmusDv:0000051': 'MmusDv:0000152','MmusDv:0000052': 'MmusDv:0000154',
    'MmusDv:0000053': 'MmusDv:0000155','MmusDv:0000054': 'MmusDv:0000156','MmusDv:0000055': 'MmusDv:0000157',
    'MmusDv:0000056': 'MmusDv:0000158','MmusDv:0000057': 'MmusDv:0000159','MmusDv:0000058': 'MmusDv:0000160',
    'MmusDv:0000059': 'MmusDv:0000161','MmusDv:0000061': 'MmusDv:0000136','MmusDv:0000065': 'MmusDv:0000162',
    'MmusDv:0000066': 'MmusDv:0000163','MmusDv:0000067': 'MmusDv:0000164','MmusDv:0000068': 'MmusDv:0000165',
    'MmusDv:0000070': 'MmusDv:0000166','MmusDv:0000071': 'MmusDv:0000167','MmusDv:0000072': 'MmusDv:0000168',
    'MmusDv:0000073': 'MmusDv:0000169','MmusDv:0000074': 'MmusDv:0000170','MmusDv:0000076': 'MmusDv:0000134',
    'MmusDv:0000096': 'MmusDv:0000138','MmusDv:0000098': 'MmusDv:0000171','MmusDv:0000099': 'MmusDv:0000172',
    'MmusDv:0000100': 'MmusDv:0000173','MmusDv:0000101': 'MmusDv:0000174','MmusDv:0000102': 'MmusDv:0000175',
    'MmusDv:0000113': 'MmusDv:0000177'
}

deprecated_donors = pd.DataFrame(adata.obs[
    (adata.obs['development_stage_ontology_term_id'].isin(deprecated_dev)) &
    (adata.obs['development_stage_ontology_term_id'].isin(auto_migrate) == False)
    ][['donor_id','development_stage_ontology_term_id']].value_counts())

if not deprecated_donors.empty:
    report('These donors have a now-deprecated development_stage term.\n'\
    'They should be reported in #825 with the term to use from the current ontology version.',
    'WARNING')
    report('https://github.com/chanzuckerberg/single-cell-curation/issues/825\n')
    display(deprecated_donors)

**Check human donors' reported sex**<br>
The metric (x) used to determine sex is the ratio of male to female raw expression counts summed across all genes found in sex_analysis_genes.json. 
- Donor is considered **male:** *if x > 0.35*
- Donor is considered **female:** *if x < 0.05*
- Donor is considered **unknown:** *if 0.05 < x < 0.35*

In [ ]:
donor_sex_df = evaluate_donors_sex(adata)

if donor_sex_df is not None:
    inconsistencies = donor_sex_df[donor_sex_df['scRNAseq_sex'] != donor_sex_df['author_annotated_sex']].sort_values('donor_id')
    if not inconsistencies.empty:
        report('donor sex metadata inconsistencies', 'ERROR')
        display(inconsistencies)
    else:
        report('donor sex metadata is consistent', 'GOOD')

# var
**Check for Ensembl IDs, redundant fields, etc.**<br>
**Check for schema fields**

In [ ]:
adata.var

In [ ]:
if len(adata.var) < 15000:
    report('Less than 15k genes present','ERROR')
elif len(adata.var) < 20000:
    report('Less than 20k genes present','WARNING')

**Similar checks for raw.var, if present**

In [ ]:
if adata.raw:
    display(adata.raw.var)

# Validate
**Determine the embedding by which to plot**\
May need to overwrite if the first obsm is not informative

In [ ]:
default_embedding = adata.uns.get('default_embedding', pick_embed(adata.obsm_keys()))
default_embedding

**Plot by multiple genes using the normalized counts**<br>
*It is best to get a list of genes relevant to the specific data from the contributor/publication*

In [ ]:
symbols = [
    'CD34',
    'IGLL1',
    'TRGC2',
    'CCR9',
    'CCR7',
    'HIVEP3',
    'TOX2',
    'RAG1',
    'RAG2',
    'PCNA',
    'CDK1'
]

ensg_list = symbols_to_ids(symbols, adata.var)

In [ ]:
sc.pl.embedding(adata, basis=default_embedding, color=ensg_list, use_raw=False)

**Compare with the same genes using the raw counts to confirm they are correlated**

In [ ]:
if adata.raw:
    sc.pl.embedding(adata, basis=default_embedding, color=ensg_list, use_raw=True)

**Additionally, you could compare dotplots of those genes in each cell population**\
*This will scale all genes based on the max range of any gene so 1 gene with high values may make others difficult to distinguish*

In [ ]:
sc.pl.dotplot(adata, ensg_list, cellpop_field, use_raw=False)

In [ ]:
if adata.raw:
    sc.pl.dotplot(adata, ensg_list, cellpop_field, use_raw=True)

**If it is spatial data, test if the image and X_spatial embeddings enable scanpy use**

In [ ]:
if adata.obs['assay_ontology_term_id'].unique()[0] == 'EFO:0010961' and adata.uns['spatial']['is_single'] == True:
    plot_vis(adata, cellpop_field)
    if remove_colors:
        del adata.uns[f'{cellpop_field}_colors']

**If submitted as an HCA Dataset, check for additional HCA standards**

In [ ]:
# hca_errors = validate_hca(adata)
# if hca_errors:
#    for e in hca_errors:
#        report(e, 'ERROR')
# else:
#    report('meets HCA Tier 1 requirements')

**If updates have been made, write the revised file**\
*`compression='gzip'` is critical here to keep the file size down*\
**Run the CELLxGENE validator on the revised file**\
*This is the same as running `cellxgene-schema validate <file>` in the terminal*

In [ ]:
# new_one = file.replace('.h5ad','_revised.h5ad')
# adata.write(filename=new_one, compression='gzip')

# valid = validate(new_one)